# README

This notebook implements an analysis pipeline for comparing gene expression
levels in non‑switch genomic regions between liver and brain tissues during
mouse embryonic development. By focusing on compartment‑stable (non‑switch) 
regions, we can evaluate whether genes located in the same nuclear compartment 
exhibit different expression patterns across tissues.

## Workflow

1. **Data preparation**  
    - Load a reference gene location BED file and merge transcripts so that
      each gene is represented by its minimal start and maximal end.
    - Read a preprocessed single‑cell RNA‑seq AnnData file, filter for high
      quality cells.

2. **E1 value processing**  
    - For each sample/timepoint, read the E1 eigenvector data for liver and
      brain.
    - Compute the absolute difference between liver and brain E1 values,
      identify the top 10 % most divergent bins (switch regions) and the
      remaining bins (non‑switch regions).

3. **Gene extraction and expression calculation**  
    - Map non‑switch bins to genes using genomic coordinates.
    - Subset the expression matrix to those genes and compute the mean
      expression per gene in liver and brain cells separately.

4. **Statistical testing and plotting**  
    - Perform a Kolmogorov–Smirnov test on the gene‑wise expression
      distributions between tissues.
    - Generate joint density plots of log‑transformed mean expression vs.
      E1 value, colored by tissue, and save them to a PDF.

## Inputs and outputs

- **Inputs**:  
  - `refgene.bed` gene locations.  
  - Eigenvector CSVs (`*_cis_eigs.csv`) for each sample and tissue.  
  - Single‑cell expression `Mouse_embryo_all_stage.h5ad`.

- **Outputs**:  
  - CSV files listing switch ranges per sample.  
  - PDF of KDE plots for non‑switch gene expression.  
  - Console logs of KS test p‑values.

This notebook is intended for reproducible analysis of compartmental gene
expression differences and can be adapted to other tissue comparisons or
developmental stages.

In [44]:
import scanpy as sc
import numpy as np
import pandas as pd
from scipy import stats

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
mpl.rcParams['pdf.fonttype'] = 42


e1_dir = '/home/goubo/CRICK/CRICK/spaceA/higashi_v2/higashi/'
exp_dir = '/home/goubo/CRICK/scpaRNA/stereo/processeddata/'
save_dir = '/home/xuyuetong/CRICK_Data_v3/Paper_Fig/NoSwitchRange_Expression/'
geneloc_dir = '/home/goubo/CRICK/CRICK/spaceA/bulkhicres/com_switch/' \
'dchic_E13.5brain_VS_E13.5liver/DifferentialResult/dchic_E13.5brain_VS_E13.5liver/geneEnrichment/'

tissue_list = ['Liver', 'Brain']
sample_list = ['E11.5L1', 'E11.5L2', 'E12.5L5', 'E12.5L6', 'E13.5C1', 'E13.5C4', 'E13.5C6', 'E14.5F5', 'E14.5F6']

In [6]:
def extract_gene_by_range(gene_loc_df, genome_range_df):
    gene_list = []
    for _, row in genome_range_df.iterrows():
        chr_id = row['chrom']
        start = row['start']
        end = row['end']
        matched_genes = gene_loc_df[(gene_loc_df['chrom'] == chr_id) & 
                                    (gene_loc_df['start'] >= start) & 
                                    (gene_loc_df['end'] <= end)]
        if len(matched_genes) > 0:
            gene_list.extend(matched_genes['gene_name'].tolist())
    return gene_list

In [ ]:
# Gene locations may have different transcripts, so we take the minimum start and maximum end position for each gene to merge the gene location information.

gene_loc_path = '{0}refgene.bed'.format(geneloc_dir)
gene_loc_df = pd.read_csv(gene_loc_path, header=None, index_col=None, sep='\t')
gene_loc_df.columns = ['chrom', 'start', 'end', 'gene_name']
gene_loc_merge_df = gene_loc_df.groupby(['chrom', 'gene_name']).agg(
    start=('start', 'min'),
    end=('end', 'max')).reset_index()
gene_loc_merge_df = gene_loc_merge_df[['chrom', 'start', 'end', 'gene_name']]
gene_loc_merge_df.sort_values(by=['chrom', 'start'], inplace=True, ignore_index=True)
print(gene_loc_merge_df)

      chrom     start       end      gene_name
0      chr1   3214482   3671498           Xkr4
1      chr1   4119866   4409241            Rp1
2      chr1   4490928   4497354          Sox17
3      chr1   4773200   4785726         Mrpl15
4      chr1   4807823   4846735         Lypla1
...     ...       ...       ...            ...
25324  chrY  88534803  88578064        Gm20819
25325  chrY  89328651  89330983   LOC115489301
25326  chrY  89998183  90000531        Gm20879
25327  chrY  90751140  90839406  G530011O06Rik
25328  chrY  90785442  90816465          Erdr1

[25329 rows x 4 columns]


In [13]:
# Prepare Expression Data

exp_path = '{0}Mouse_embryo_all_stage.h5ad'.format(exp_dir)
exp_adata = sc.read_h5ad(exp_path)
high_quality = exp_adata.obs["total_counts"] > 2000
exp_adata = exp_adata[high_quality, :]
print(exp_adata)

View of AnnData object with n_obs × n_vars = 478354 × 23761
    obs: 'annotation', 'timepoint', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'annotation_colors'
    obsm: 'spatial'
    layers: 'count'


In [ ]:
plot_path = '{0}Kdeplot_NonSwitch_Expression.pdf'.format(save_dir)

with PdfPages(plot_path) as pdf:
    for s, sample_id in enumerate(sample_list):

        # filter Top10% Diff-E1 range (Liver vs Brain)
        e1_liver_path = '{0}{1}/{1}_fasthigashi_leiden_anno_man_{2}.cis_eigs.csv'.format(e1_dir, sample_id, "Liver")
        e1_liver_df = pd.read_csv(e1_liver_path, header=0, index_col=0, sep=',')
        e1_liver_df['start_bin'] = e1_liver_df['start'] // 1_000_000
        e1_liver_df['start_bin'] = e1_liver_df['chrom'] + '_' + e1_liver_df['start_bin'].apply(str)
        e1_liver_array = e1_liver_df['E1'].values

        e1_brain_path = '{0}{1}/{1}_fasthigashi_leiden_anno_man_{2}.cis_eigs.csv'.format(e1_dir, sample_id, "Brain")
        e1_brain_df = pd.read_csv(e1_brain_path, header=0, index_col=0, sep=',')
        e1_brain_df['start_bin'] = e1_brain_df['start'] // 1_000_000
        e1_brain_df['start_bin'] = e1_brain_df['chrom'] + '_' + e1_brain_df['start_bin'].apply(str)
        e1_brain_array = e1_brain_df['E1'].values

        e1_diff = abs(e1_brain_array - e1_liver_array)
        e1_diff = np.nan_to_num(e1_diff)
        top_count = int(np.ceil(len(e1_diff) * 0.1))
        switch_indices = np.argsort(e1_diff)[::-1][:top_count]
        noswitch_indices = np.argsort(e1_diff)[::-1][top_count:]
        switch_df = e1_liver_df.iloc[switch_indices, :]
        noswitch_df = e1_liver_df.iloc[noswitch_indices, :]
        switch_df.to_csv('{0}SwitchRange_{1}.csv'.format(save_dir, sample_id), index=False)

        # extract gene by noswitch range
        noswitch_gene = extract_gene_by_range(gene_loc_merge_df, noswitch_df)
        noswitch_gene = [gene for gene in noswitch_gene if gene in exp_adata.var_names]
        noswitch_geneloc_df = gene_loc_merge_df[gene_loc_merge_df['gene_name'].isin(noswitch_gene)].copy()
        noswitch_geneloc_df['start_bin'] = noswitch_geneloc_df['start'] // 1_000_000
        noswitch_geneloc_df['start_bin'] = noswitch_geneloc_df['chrom'] + '_' + noswitch_geneloc_df['start_bin'].apply(str)
        

        # extract noswitch-gene expression
        time = sample_id[:5]
        liver_exp_adata = exp_adata[(exp_adata.obs['timepoint'] == time) & (exp_adata.obs['annotation'] == 'Liver')]
        liver_noswitchgene_exp = liver_exp_adata[:, noswitch_gene]
        if hasattr(liver_noswitchgene_exp.X, 'toarray'):
            liver_column_means = np.mean(liver_noswitchgene_exp.X.toarray(), axis=0)
        else:
            liver_column_means = np.mean(liver_noswitchgene_exp.X, axis=0)
        liver_noswitchgene_expmeans = pd.DataFrame({'gene_name':liver_noswitchgene_exp.var_names, 'exp_mean': liver_column_means, 'tissue_id': 'Liver'})
        liver_noswitchgene_expmeans = liver_noswitchgene_expmeans.merge(noswitch_geneloc_df, on=['gene_name'], how='left')
        bin_e1_liver_dict = dict(e1_liver_df[['start_bin', 'E1']].values)
        liver_noswitchgene_expmeans['E1'] = liver_noswitchgene_expmeans['start_bin'].map(bin_e1_liver_dict)

        brain_exp_adata = exp_adata[(exp_adata.obs['timepoint'] == time) & (exp_adata.obs['annotation'] == 'Brain')]
        brain_noswitchgene_exp = brain_exp_adata[:, noswitch_gene]
        if hasattr(brain_noswitchgene_exp.X, 'toarray'):
            brain_column_means = np.mean(brain_noswitchgene_exp.X.toarray(), axis=0)
        else:
            brain_column_means = np.mean(brain_noswitchgene_exp.X, axis=0)
        brain_noswitchgene_expmeans = pd.DataFrame({'gene_name':brain_noswitchgene_exp.var_names, 'exp_mean': brain_column_means, 'tissue_id': 'Brain'})
        brain_noswitchgene_expmeans = brain_noswitchgene_expmeans.merge(noswitch_geneloc_df, on=['gene_name'], how='left')
        bin_e1_brain_dict = dict(e1_brain_df[['start_bin', 'E1']].values)
        brain_noswitchgene_expmeans['E1'] = brain_noswitchgene_expmeans['start_bin'].map(bin_e1_brain_dict)

        ks_statistic, ks_pvalue = stats.ks_2samp(liver_column_means, brain_column_means)
        print(f"{sample_id}\tKS test: {ks_pvalue}")
        
        noswitchgene_expmeans = pd.concat([liver_noswitchgene_expmeans, brain_noswitchgene_expmeans], ignore_index=True)
        noswitchgene_expmeans['log_exp_mean'] = noswitchgene_expmeans['exp_mean'].apply(lambda x: np.log1p(x))
        
        # plot
        plt.figure(figsize=(4, 4))
        j = sns.jointplot(data=noswitchgene_expmeans, x="log_exp_mean", y="E1", hue="tissue_id", kind="kde")
        plt.title(sample_id, y=0.95)
        plt.xlim(-0.15, 1.00)
        plt.xlabel('log(Expression Mean)')
        plt.tight_layout()
        pdf.savefig()
        plt.close()

       